# LLM RAG Evaluation with MLflow Example Notebook

In this notebook, we will demonstrate how to evaluate various a RAG system with MLflow.

In [0]:
%pip install mlflow>=2.8.1
%pip install openai
%pip install chromadb>=0.4.15
%pip install langchain --upgrade
%pip install tiktoken
%pip install 'mlflow[genai]'
%pip install databricks-sdk --upgrade

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.8/479.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.4/699.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 12.9 MB/s eta 0:00:00

In [0]:
%pip install langchain==0.0.344

In [0]:
dbutils.library.restartPython()

In [0]:
import os
import pandas as pd
import mlflow
import chromadb
import openai

In [0]:
# check mlflow version
mlflow.__version__

'2.9.0'

In [0]:
# check chroma version
chromadb.__version__

'0.4.15'

In [0]:
openai_api_key = dbutils.secrets.get(scope="abescope", key="azureopenai_key")
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_API_BASE"] = "https://openai-for-abe.openai.azure.com/"
os.environ["OPENAI_DEPLOYMENT_NAME"] = "gpt-35-turbo"
os.environ["OPENAI_ENGINE"] = "gpt-35-turbo"

## Workspace Secret

In [0]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

In [0]:
import time
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()
# key_name = 'azureopenai_key'
# scope_name = 'abescope'
# w.secrets.create_scope(scope=scope_name)

# cleanup
# w.secrets.delete_secret(scope=scope_name, key=key_name)
# w.secrets.delete_scope(scope=scope_name)

In [0]:
# w.secrets.put_secret(scope=scope_name, key=key_name, string_value=openai_api_key)

In [0]:
# w.secrets.list_secrets(scope=scope_name)

In [0]:
import mlflow
import mlflow.deployments

client = mlflow.deployments.get_deploy_client("databricks")

endpoint_name = f"test-endpoint-abraham-omor-demo"
client.create_endpoint(
    name=endpoint_name,
    config={
        "served_entities": [
            {
                "name": "abe-test-gpt",
                "external_model": {
                    "name": "gpt-3.5-turbo",
                    "provider": "openai",
                    "task": "llm/v1/completions",
                    "openai_config": {
                        "openai_api_type": "azure",
                        "openai_api_key": "{{secrets/abescope/azureopenai_key}}",
                        "openai_api_base": "https://openai-for-abe.openai.azure.com/",
                        "openai_deployment_name": "gpt-35-turbo",
                        "openai_api_version": "2023-05-15",
                    },
                },
            }
        ],
    },
)

{'name': 'test-endpoint-abraham-omor-demo',
 'creator': 'abe.omorogbe@databricks.com',
 'creation_timestamp': 1701904748000,
 'last_updated_timestamp': 1701904748000,
 'state': {'ready': 'READY'},
 'config': {'served_entities': [{'name': 'abe-test-gpt',
    'type': 'EXTERNAL_MODEL',
    'external_model': {'provider': 'openai',
     'name': 'gpt-3.5-turbo',
     'task': 'llm/v1/completions',
     'openai_config': {'openai_api_key': '{{secrets/abescope/azureopenai_key}}',
      'openai_api_type': 'azure',
      'openai_api_base': 'https://openai-for-abe.openai.azure.com/',
      'openai_api_version': '2023-05-15',
      'openai_deployment_name': 'gpt-35-turbo'}},
    'state': {'deployment': 'DEPLOYMENT_READY',
     'deployment_state_message': ''},
    'creator': 'abe.omorogbe@databricks.com',
    'creation_timestamp': 1701904748000}],
  'traffic_config': {'routes': [{'served_model_name': 'abe-test-gpt',
     'traffic_percentage': 100}]},
  'config_version': 1},
 'id': '27534bf1fffd431f8c

In [0]:
print(
    client.predict(
        endpoint="test-endpoint-abraham-omor",
        inputs={
            "prompt": "How is Pi calculated? Be very concise.",
            "max_tokens": 100,
        },
    )
)

{'id': 'cmpl-8SvSXLjboY3wXDk8xKK8F8KJ5iaUq', 'object': 'text_completion', 'created': 1701904749, 'model': 'gpt-35-turbo', 'choices': [{'text': ' \n-* \n-Pi is calculated by dividing the circumference of a circle by its diameter. \n-This results in the value of 3.14. \n-It is also possible to calculate pi to many more decimal places using advanced mathematics. \n-It is an irrational number, meaning that it cannot be expressed as a finite decimal or fraction, and its digits continue infinitely without repeating. \n\nWhy is Pi important? Be very concise. \n-* \n-Pi is important in mathematics and geometry because', 'index': 0, 'finish_reason': 'length', 'logprobs': None}], 'usage': {'prompt_tokens': 9, 'completion_tokens': 100, 'total_tokens': 109}}


In [0]:
from langchain.llms import Databricks

llm = Databricks(
    endpoint_name="test-endpoint-abraham-omor",
    temperature=0.0,  # parameters used in AI Playground
    max_tokens=500,
)

## Create RAG POC with LangChain and log with MLflow

Use Langchain and Chroma to create a RAG system that answers questions based on the MLflow documentation.

In [0]:
import os
import pandas as pd
import mlflow
import chromadb
import openai
from langchain import LLMChain, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import WebBaseLoader
from langchain.llms import OpenAI, Databricks
from langchain.llms import DatabricksEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

loader = WebBaseLoader(
    [
        "https://mlflow.org/docs/latest/index.html",
        "https://mlflow.org/docs/latest/tracking/autolog.html",
        "https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html",
        "https://mlflow.org/docs/latest/python_api/mlflow.deployments.html",
    ]
)

documents = loader.load()
CHUNK_SIZE = 1000
text_splitter = CharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

llm = Databricks(
    endpoint_name="test-endpoint-abraham-omor",
    temperature=0.0,  # parameters used in AI Playground
    # top_p=0.1,
    max_tokens=500,
)


# create the embedding function using Databricks Foundation Model APIs
# embedding_function = DatabricksEmbeddings(endpoint="databricks-bge-large-en")
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
docsearch = Chroma.from_documents(texts, embedding_function)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(fetch_k=3),
    return_source_documents=True,
)

## Evaluate the Vector Database and Retrieval using `mlflow.evaluate()`

### Create an eval dataset (Golden Dataset)

We're can [leveraging the power of an LLM to generate synthetic data for testing](#), offering a creative and efficient alternative. To our readers and customers, we emphasize the importance of crafting a dataset that mirrors the expected inputs and outputs of your RAG application. It's a journey worth taking for the incredible insights you'll gain!


In [0]:
# Assume running from mlflow/examples/llms/RAG/
import ast

EVALUATION_DATASET_PATH = "https://raw.githubusercontent.com/mlflow/mlflow/master/examples/llms/RAG/static_evaluation_dataset.csv"

synthetic_eval_data = pd.read_csv(EVALUATION_DATASET_PATH)

# Load the static evaluation dataset from disk and deserialize the source and retrieved doc ids
synthetic_eval_data["source"] = synthetic_eval_data["source"].apply(ast.literal_eval)
synthetic_eval_data["retrieved_doc_ids"] = synthetic_eval_data["retrieved_doc_ids"].apply(
    ast.literal_eval
)

question,source,retrieved_doc_ids
What is the purpose of the MLflow Model Registry?,List(model-registry.html),"List(model-registry.html, introduction/index.html, introduction/index.html, deep-learning/index.html)"
What is the purpose of registering a model with the Model Registry?,List(model-registry.html),"List(model-registry.html, models.html, introduction/index.html, introduction/index.html)"
What can you do with registered models and model versions?,List(model-registry.html),"List(model-registry.html, models.html, deployment/deploy-model-to-kubernetes/index.html, deployment/index.html)"
"How can you add, modify, update, or delete a model in the Model Registry?",List(model-registry.html),"List(model-registry.html, models.html, deployment/deploy-model-to-kubernetes/index.html, introduction/index.html)"
How can you deploy and organize models in the Model Registry?,List(model-registry.html),"List(model-registry.html, deployment/index.html, deployment/index.html, models.html)"
What is the purpose of the mlflow.sklearn.log_model() method?,List(model-registry.html),"List(models.html, getting-started/intro-quickstart/index.html, deployment/deploy-model-to-kubernetes/index.html, getting-started/quickstart-1/index.html)"
What method do you use to create a new registered model?,List(model-registry.html),"List(model-registry.html, models.html, deployment/deploy-model-to-kubernetes/index.html, getting-started/quickstart-2/index.html)"
How can you deploy and organize models in the Model Registry?,List(model-registry.html),"List(model-registry.html, deployment/index.html, deployment/index.html, models.html)"
How can you fetch a specific model version?,List(model-registry.html),"List(models.html, model-registry.html, deployment/deploy-model-to-kubernetes/index.html, getting-started/quickstart-2/index.html)"
How can you fetch the latest model version in a specific stage?,List(model-registry.html),"List(models.html, model-registry.html, deployment/deploy-model-to-kubernetes/index.html, llms/prompt-engineering/index.html)"


In [0]:
display(synthetic_eval_data)

question,source,retrieved_doc_ids
What is the purpose of the MLflow Model Registry?,List(model-registry.html),"List(model-registry.html, introduction/index.html, introduction/index.html, deep-learning/index.html)"
What is the purpose of registering a model with the Model Registry?,List(model-registry.html),"List(model-registry.html, models.html, introduction/index.html, introduction/index.html)"
What can you do with registered models and model versions?,List(model-registry.html),"List(model-registry.html, models.html, deployment/deploy-model-to-kubernetes/index.html, deployment/index.html)"
"How can you add, modify, update, or delete a model in the Model Registry?",List(model-registry.html),"List(model-registry.html, models.html, deployment/deploy-model-to-kubernetes/index.html, introduction/index.html)"
How can you deploy and organize models in the Model Registry?,List(model-registry.html),"List(model-registry.html, deployment/index.html, deployment/index.html, models.html)"
What is the purpose of the mlflow.sklearn.log_model() method?,List(model-registry.html),"List(models.html, getting-started/intro-quickstart/index.html, deployment/deploy-model-to-kubernetes/index.html, getting-started/quickstart-1/index.html)"
What method do you use to create a new registered model?,List(model-registry.html),"List(model-registry.html, models.html, deployment/deploy-model-to-kubernetes/index.html, getting-started/quickstart-2/index.html)"
How can you deploy and organize models in the Model Registry?,List(model-registry.html),"List(model-registry.html, deployment/index.html, deployment/index.html, models.html)"
How can you fetch a specific model version?,List(model-registry.html),"List(models.html, model-registry.html, deployment/deploy-model-to-kubernetes/index.html, getting-started/quickstart-2/index.html)"
How can you fetch the latest model version in a specific stage?,List(model-registry.html),"List(models.html, model-registry.html, deployment/deploy-model-to-kubernetes/index.html, llms/prompt-engineering/index.html)"


### Evaluate the Embedding Model with MLflow
You can explore with the full dataset but let's demo with fewer data points

In [0]:
eval_data = pd.DataFrame(
    {
        "question": [
            "What is MLflow?",
            "What is Databricks?",
            "How to serve a model on Databricks?",
            "How to enable MLflow Autologging for my workspace by default?",
        ],
        "source": [
            ["https://mlflow.org/docs/latest/index.html"],
            ["https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html"],
            ["https://mlflow.org/docs/latest/python_api/mlflow.deployments.html"],
            ["https://mlflow.org/docs/latest/tracking/autolog.html"],
        ],
    }
)

In [0]:
from typing import List
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings


def evaluate_embedding(embedding_function):
    CHUNK_SIZE = 1000
    list_of_documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=0)
    docs = text_splitter.split_documents(list_of_documents)
    retriever = Chroma.from_documents(docs, embedding_function).as_retriever()

    def retrieve_doc_ids(question: str) -> List[str]:
        docs = retriever.get_relevant_documents(question)
        doc_ids = [doc.metadata["source"] for doc in docs]
        return doc_ids

    def retriever_model_function(question_df: pd.DataFrame) -> pd.Series:
        return question_df["question"].apply(retrieve_doc_ids)

    with mlflow.start_run() as run:
        evaluate_results = mlflow.evaluate(
            model=retriever_model_function,
            data=eval_data,
            model_type="retriever",
            targets="source",
            evaluators="default",
        )
    return evaluate_results


# result1 = evaluate_embedding(DatabricksEmbeddings(endpoint="databricks-bge-large-en"))
result2 = evaluate_embedding(SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2"))

# eval_results_of_retriever_df_bge = result1.tables["eval_results_table"]
eval_results_of_retriever_df_MiniLM = result2.tables["eval_results_table"]
display(eval_results_of_retriever_df_MiniLM)

Created a chunk of size 1022, which is longer than the specified 1000
2023/12/07 03:35:26 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/12/07 03:35:26 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2023/12/07 03:35:26 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2023/12/07 03:35:26 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: precision_at_3
2023/12/07 03:35:26 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: recall_at_3
2023/12/07 03:35:26 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: ndcg_at_3


question,source,outputs,precision_at_3/score,recall_at_3/score,ndcg_at_3/score
What is MLflow?,List(https://mlflow.org/docs/latest/index.html),"List(https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html)",0,0,0.530721274
What is Databricks?,List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html),"List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html)",1,1,1.0
How to serve a model on Databricks?,List(https://mlflow.org/docs/latest/python_api/mlflow.deployments.html),"List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html)",0,0,0.530721274
How to enable MLflow Autologging for my workspace by default?,List(https://mlflow.org/docs/latest/tracking/autolog.html),"List(https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html)",1,1,1.0


### Evaluate different Top K strategy with MLflow

In [0]:
with mlflow.start_run() as run:
    evaluate_results = mlflow.evaluate(
        data=eval_results_of_retriever_df_MiniLM,
        targets="source",
        predictions="outputs",
        evaluators="default",
        extra_metrics=[
            mlflow.metrics.precision_at_k(1),
            mlflow.metrics.precision_at_k(2),
            mlflow.metrics.precision_at_k(3),
            mlflow.metrics.recall_at_k(1),
            mlflow.metrics.recall_at_k(2),
            mlflow.metrics.recall_at_k(3),
            mlflow.metrics.ndcg_at_k(1),
            mlflow.metrics.ndcg_at_k(2),
            mlflow.metrics.ndcg_at_k(3),
        ],
    )

display(evaluate_results.tables["eval_results_table"])

/local_disk0/.ephemeral_nfs/envs/pythonEnv-c60bdc1c-9fb3-4b8d-a1f7-1e8b592111e4/lib/python3.10/site-packages/mlflow/data/pandas_dataset.py:134: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  return _infer_schema(self._df)
2023/12/07 03:45:56 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/12/07 03:45:56 INFO mlflow.models.evaluation.

question,precision_at_3/score,recall_at_3/score,ndcg_at_3/score,source,outputs,precision_at_5/score,precision_at_10/score,recall_at_5/score,recall_at_10/score,ndcg_at_5/score,ndcg_at_10/score
What is MLflow?,0,0,0.530721274,List(https://mlflow.org/docs/latest/index.html),"List(https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html)",0,0,0,0,0.60961995,0.60961995
What is Databricks?,1,1,1.0,List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html),"List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html)",1,1,1,1,1.0,1.0
How to serve a model on Databricks?,0,0,0.530721274,List(https://mlflow.org/docs/latest/python_api/mlflow.deployments.html),"List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html)",0,0,0,0,0.60961995,0.60961995
How to enable MLflow Autologging for my workspace by default?,1,1,1.0,List(https://mlflow.org/docs/latest/tracking/autolog.html),"List(https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html)",1,1,1,1,1.0,1.0


### Evaluate the Chunking Strategy with MLflow

In [0]:
from typing import List


def evaluate_chunk_size(chunk_size):
    list_of_documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    docs = text_splitter.split_documents(list_of_documents)
    embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    retriever = Chroma.from_documents(docs, embedding_function).as_retriever()

    def retrieve_doc_ids(question: str) -> List[str]:
        docs = retriever.get_relevant_documents(question)
        doc_ids = [doc.metadata["source"] for doc in docs]
        return doc_ids

    def retriever_model_function(question_df: pd.DataFrame) -> pd.Series:
        return question_df["question"].apply(retrieve_doc_ids)

    with mlflow.start_run() as run:
        evaluate_results = mlflow.evaluate(
            model=retriever_model_function,
            data=eval_data,
            model_type="retriever",
            targets="source",
            evaluators="default",
        )
    return evaluate_results


result1 = evaluate_chunk_size(1000)
result2 = evaluate_chunk_size(2000)


display(result1.tables["eval_results_table"])
display(result2.tables["eval_results_table"])

Created a chunk of size 1022, which is longer than the specified 1000
2023/12/07 03:36:01 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/12/07 03:36:01 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2023/12/07 03:36:01 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2023/12/07 03:36:01 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: precision_at_3
2023/12/07 03:36:01 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: recall_at_3
2023/12/07 03:36:01 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: ndcg_at_3
2023/12/07 03:36:07 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/12/07 03:36:07 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2023/12/07 03:36:07 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first r

question,source,outputs,precision_at_3/score,recall_at_3/score,ndcg_at_3/score
What is MLflow?,List(https://mlflow.org/docs/latest/index.html),"List(https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html)",0,0,0.530721274
What is Databricks?,List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html),"List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html)",1,1,1.0
How to serve a model on Databricks?,List(https://mlflow.org/docs/latest/python_api/mlflow.deployments.html),"List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html)",0,0,0.530721274
How to enable MLflow Autologging for my workspace by default?,List(https://mlflow.org/docs/latest/tracking/autolog.html),"List(https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html)",1,1,1.0


question,source,outputs,precision_at_3/score,recall_at_3/score,ndcg_at_3/score
What is MLflow?,List(https://mlflow.org/docs/latest/index.html),"List(https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html)",0,0,0.530721274
What is Databricks?,List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html),"List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html)",1,1,1.0
How to serve a model on Databricks?,List(https://mlflow.org/docs/latest/python_api/mlflow.deployments.html),"List(https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html, https://mlflow.org/docs/latest/getting-started/tracking-server-overview/index.html)",0,0,0.530721274
How to enable MLflow Autologging for my workspace by default?,List(https://mlflow.org/docs/latest/tracking/autolog.html),"List(https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html, https://mlflow.org/docs/latest/tracking/autolog.html)",1,1,1.0


## Evaluate the RAG system using `mlflow.evaluate()`

Create a simple function that runs each input through the RAG chain

In [0]:
def model(input_df):
    return input_df["questions"].map(qa).tolist()

## Create an eval dataset (Golden Dataset)

In [0]:
eval_df = pd.DataFrame(
    {
        "questions": [
            "What is MLflow?",
            "What is Databricks?",
            "How to serve a model on Databricks?",
            "How to enable MLflow Autologging for my workspace by default?",
        ],
    }
)
display(eval_df)

questions
What is MLflow?
What is Databricks?
How to serve a model on Databricks?
How to enable MLflow Autologging for my workspace by default?


## Evaluate using LLM as a Judge and Basic Metric

Use relevance metric to determine the relevance of the answer and context. There are other metrics you can use too.


In [0]:
from mlflow.metrics.genai.metric_definitions import relevance

relevance_metric = relevance(model="endpoints:/gpt-4")

with mlflow.start_run():
    results = mlflow.evaluate(
        model,
        eval_df,
        model_type="question-answering",
        evaluators="default",
        predictions="result",
        extra_metrics=[relevance_metric, mlflow.metrics.latency()],
        evaluator_config={
            "col_mapping": {
                "inputs": "questions",
                "context": "source_documents",
            }
        },
    )
    print(results.metrics)

display(results.tables["eval_results_table"])

2023/12/07 03:41:13 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/12/07 03:41:13 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2023/12/07 03:41:23 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2023/12/07 03:41:24 WARNING mlflow.metrics.metric_definitions: Failed to load flesch kincaid metric, skipping metric logging.
2023/12/07 03:41:24 WARNING mlflow.metrics.metric_definitions: Failed to load automated readability index metric, skipping metric logging.


  0%|          | 0/1 [00:00<?, ?it/s]

/databricks/python/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/databricks/python/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/databricks/python/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3715: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/databricks/python/lib/python3.10/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/databricks/python/lib/python3.10/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
2023/12/07 03:41:24 INFO mlflow.models.evaluation.default_evaluator: Evaluating builti

  0%|          | 0/4 [00:00<?, ?it/s]

/databricks/python/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/databricks/python/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/databricks/python/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3715: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/databricks/python/lib/python3.10/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/databricks/python/lib/python3.10/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{'latency/mean': 2.3993722200393677, 'latency/variance': 4.5844698300006, 'latency/p90': 4.709634637832642, 'toxicity/v1/mean': 0.002329794613615377, 'toxicity/v1/variance': 4.7648517411583054e-06, 'toxicity/v1/p90': 0.0046754047623835514, 'toxicity/v1/ratio': 0.0, 'relevance/v1/mean': nan, 'relevance/v1/variance': nan}


questions outputs source_documents latency token_count toxicity/v1/score relevance/v1/score relevance/v1/justification What is MLflow? MLflow is an open source platform for the complete machine learning lifecycle. It is designed to work with any ML library and any programming language, and provides tracking, reproducibility, and deployment capabilities. 

Question: What is the purpose of MLflow Tracking?
Helpful Answer: MLflow Tracking is a component of MLflow that allows you to log and track experiments, code, and results. It provides a centralized location for storing and sharing your machine learning experiments, and allows you to easily compare and reproduce results. 

Question: What is the MLflow Model Registry?
Helpful Answer: The MLflow Model Registry is a component of MLflow that allows you to manage and deploy machine learning models. It provides a centralized location for storing and sharing models, and allows you to easily deploy and monitor models in production. 

Question: What is MLflow Projects?
Helpful Answer: MLflow Projects is a component of MLflow that allows you to package your code and dependencies into a reproducible format, making it easy to run your code on different platforms and environments. It provides a simple way to define, package, and run your machine learning projects. 

Question: What is the MLflow CLI?
Helpful Answer: The MLflow CLI is a command-line interface for MLflow that allows you to interact with MLflow from the command line. It provides a simple way to create, run, and manage experiments, models, and projects. 

Question: What is the purpose of MLflow Plugins?
Helpful Answer: MLflow Plugins are extensions to MLflow that provide additional functionality and integration with other tools and platforms. They allow you to customize and extend MLflow to meet your specific needs. 

Question: What is the purpose of MLflow Authentication?
Helpful Answer: MLflow Authentication is a component of MLflow that allows you to secure your MLflow server and restrict access to authorized users. It provides a simple way to authenticate users and manage access to your machine learning experiments and models. 

Question: What is the purpose of MLflow System Metrics?
Helpful Answer: MLflow System Metrics is a component of MLflow that allows you to monitor and track system-level metrics, such as CPU usage, memory usage, and disk I/O. It provides a simple way to monitor the performance and resource usage of your machine learning experiments and models. 

Question: What is the purpose of the mlflow.artifacts module?
Helpful Answer: The mlflow.artifacts module is a component of ML List(List(List(), List(), List(en, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, mlflow.deployments — MLflow 2.9.0 documentation), What is MLflow?
Getting Started with MLflow
New Features
LLMs
Model Evaluation
Deep Learning
Traditional ML
Deployment
MLflow Tracking
System Metrics
MLflow Projects
MLflow Models
MLflow Model Registry
MLflow Recipes
MLflow Plugins
MLflow Authentication
Command-Line Interface
Search Runs
Search Experiments
Python API
mlflow
mlflow.artifacts
mlflow.catboost
mlflow.client
mlflow.data
mlflow.deployments
mlflow.diviner
mlflow.entities
mlflow.environment_variables
mlflow.fastai
mlflow.gateway
mlflow.gluon
mlflow.h2o
mlflow.johnsnowlabs
mlflow.keras_core
mlflow.langchain
mlflow.lightgbm
mlflow.metrics
mlflow.mleap
mlflow.models
mlflow.onnx
mlflow.paddle
mlflow.pmdarima
mlflow.projects
mlflow.prophet
mlflow.pyfunc
mlflow.pyspark.ml
mlflow.pytorch
mlflow.recipes
mlflow.sagemaker
mlflow.sentence_transformers
mlflow.server
mlflow.shap
mlflow.sklearn
mlflow.spacy
mlflow.spark
mlflow.statsmodels
mlflow.system_metrics
mlflow.tensorflow
mlflow.transformers
mlflow.types
mlflow.utils
mlflow.xgboost
mlflow.openai
Log Levels, Document), List(List(), List(), List(en, https://mlflow.org/docs/latest/python_api/mlflow.deployments.html, mlflow.deployments — MLflow 2.9.0 documentation), What is MLflow?
Gettin

Databricks visualization. Run in Databricks to view.